***
## План тестирования

Ваша ближайшая задача — протестировать проект, а для этого нужно спланировать, что именно нужно тестировать, а что не стоит.

Будем тестировать:

В файле *test_routes.py*:

* Главная страница доступна анонимному пользователю.

* Страница отдельной новости доступна анонимному пользователю.

* Страницы удаления и редактирования комментария доступны автору комментария.

* При попытке перейти на страницу редактирования или удаления комментария анонимный пользователь перенаправляется на страницу авторизации.

* Авторизованный пользователь не может зайти на страницы редактирования или удаления чужих комментариев (возвращается ошибка 404).

* Страницы регистрации пользователей, входа в учётную запись и выхода из неё доступны анонимным пользователям.

В файле *test_content.py*:

* Количество новостей на главной странице — не более 10.

* Новости отсортированы от самой свежей к самой старой. Свежие новости в начале списка.

* Комментарии на странице отдельной новости отсортированы от старых к новым: старые в начале списка, новые — в конце.

* Анонимному пользователю недоступна форма для отправки комментария на странице отдельной новости, а авторизованному доступна.

В файле *test_logic.py*:

* Анонимный пользователь не может отправить комментарий.

* Авторизованный пользователь может отправить комментарий.

* Если комментарий содержит запрещённые слова, он не будет опубликован, а форма вернёт ошибку.

* Авторизованный пользователь может редактировать или удалять свои комментарии.

* Авторизованный пользователь не может редактировать или удалять чужие комментарии.

Не будем тестировать:

* Непосредственно регистрацию пользователей, процесс входа в учетную запись и выхода из неё.

* Всё, что связано с админ-зоной проекта.

* Абсолютные url-адреса; обращаться к адресам будем при помощи функции `reverse('name')`.

* Какие шаблоны используются, и насколько корректно шаблон обрабатывает полученную от Django информацию. Например, если в шаблон передается список объектов, мы будем считать, что они выводятся все, а их сортировка не меняется.

* Всё то, что не попало в список «будем тестировать».

***
## Запуск и обнаружение тестов

Сейчас в Django-проекте нет ни одного теста, но модуль для тестирования уже готов к работе — и можно выполнить команду поиска и запуска тестов.

Тесты в Django запускаются командой `python manage.py test`. Django будет искать тесты в пакетах текущей директории и во всех вложенных пакетах, в файлах с именем `test*.py`. Вместо символа «звёздочка» может быть любой набор символов, например, будут найдены файлы с именами *test_models.py*, *test_content.py*, *testsomethingelse.py*.

Вот пример структуры, в которой каждый из файлов будет найден и распознан как тест. Чтобы структура выглядела полегче — из неё удалены все файлы, кроме тестов и файлов `__init__.py`.

>В директориях с тестами обязательно должен быть файл `__init__.py`

***
## Где живут тесты Django

При выполнении команды «создать новое приложение» `python manage.py startapp <app_name>` в директории приложения автоматически создаётся файл для тестов `tests.py`:

Это лучше, чем ничего, но на практике размещение всех тестов в одном файле приведёт к тому, что файл раздуется и станет плохо читаемым. 

Лучше делать так: 

1. Удалить файл *tests.py* (если его не удалить, то может возникнуть путаница между модулем *tests.py* и пакетом *tests* из следующего шага).

2. Создать в директории приложения **пакет** `tests`: создать директорию tests/ и разместить в ней пустой файл `__init__.py`.

3. В пакете `tests` создать отдельные файлы для тестов.

Есть разные подходы к тому, как называть файлы с тестами и какие тесты в каких файлах сохранять. Например, можно построить структуру файлов с тестами на основе файлов приложения: тесты для кода из *views.py* хранить в файле *test_views.py*, а тесты для *models.py* — в *test_models.py*. Следуя этой логике, в директории с тестами следует создать файлы для тестирования моделей, представлений, форм и адресов:


У такого подхода есть недостаток — обычно логика работы приложения распределена между разными файлами, и непонятно, где именно её тестировать:

1. Где тестировать доступность эндпоинта для авторизованного юзера, если эндпоинт описан в *urls.py*, а необходимость авторизации пользователя — во views.py?

2. Где тестировать шаблоны для view-классов, если они могут быть указаны и во *views.py*, и в *urls.py*?

3. Где тестировать форму, если она описывается в *forms.py*, а логика работы с ней — во *views.py*?

Есть и другая логика для структурирования тестов: их можно расположить по функциональному признаку, без жёсткой привязки к тому, в каком конкретно файле находится проверяемый код:


При этом подходе логика распределения тестов по файлам будет такой:

* *test_content* — тесты, касающиеся отображения контента. Какие данные на каких страницах отображаются, какие при этом используются шаблоны, как работает пагинатор.

* *test_logic* — тестирование бизнес-логики приложения. Как обрабатываются те или иные формы, разрешено ли создание объектов с неуникальными полями, как работает специфичная логика конкретного приложения.

* *test_routes* — тесты доступности конкретных эндпоинтов, проверка редиректов, кодов ответа, которые возвращают страницы, тестирование доступа для авторизованных или анонимных пользователей.

Именно такой схемы расположения тестов мы и будем придерживаться в уроках.

Прежде, чем переходить к следующему уроку, создайте в приложении news проекта YaNews пакет `tests`, а в нём — пять пустых файлов:

* `__init__.py`: этот файл указывает, что директория — это пакет Python,

* `test_content.py`,

* `test_logic.py`,

* `test_routes.py`,

* `test_trial.py`: это файл для экспериментов с тестами.